In [2]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
from rdflib import Graph

# === Globals ===
instance_file_path = None
result_graph = None

# === Upload-Widget immer anzeigen (aber nicht immer verwenden) ===
upload_widget = widgets.FileUpload(
    accept='.ttl',
    multiple=False,
    description='Upload TTL file'
)

# === Output Widgets ===
output = widgets.Output()
shacl_output = widgets.Output()

# === Auswahlbuttons ===
option_selector = widgets.ToggleButtons(
    options=[('Use example instance', 'example'), ('Upload your own', 'upload')],
    description='Choose input:',
    style={'description_width': 'initial'}
)

# === "Continue"-Button ===
continue_button = widgets.Button(description="Continue", button_style="info")

# === Auswahlverhalten ===
def on_selection_change(change):
    output.clear_output()
    with output:
        if change['new'] == 'example':
            global instance_file_path
            instance_file_path = os.path.join("Instance_Files", "swemls-instances.ttl")
            print(f"✅ Selected example file: {instance_file_path}")
        elif change['new'] == 'upload':
            print("📁 Please upload a TTL file:")
    display(upload_widget)

option_selector.observe(on_selection_change, names='value')

# === Uploadverhalten ===
def on_file_upload(change):
    global instance_file_path
    uploaded_file = list(upload_widget.value.values())[0]
    instance_file_path = uploaded_file['metadata']['name']
    with open(instance_file_path, 'wb') as f:
        f.write(uploaded_file['content'])
    output.clear_output()
    with output:
        print(f"✅ File uploaded and saved: {instance_file_path}")

upload_widget.observe(on_file_upload, names='value')

# === "Continue"-Verhalten ===
def on_continue_clicked(b):
    output.clear_output()
    if not instance_file_path or not os.path.exists(instance_file_path):
        with output:
            print("❌ No valid file selected.")
        return
    
    with output:
        print(f"📥 Loading file: {instance_file_path}")
    
    # Beispiel: Nur laden und Tripel zählen
    g = Graph()
    try:
        g.parse(instance_file_path, format="turtle")
    except Exception as e:
        with output:
            print(f"❌ Failed to parse: {e}")
        return

    with output:
        print(f"✅ Loaded! Number of triples: {len(g)}")

continue_button.on_click(on_continue_clicked)

# === Anzeige UI ===
display(option_selector, upload_widget, output, continue_button)


ModuleNotFoundError: No module named 'rdflib'